# Hypothesis Testing
#### null hypothesis: option expiry has no effect on the trend of nifty50

In [1]:
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
import yfinance as yf
import pandas as pd

In [4]:
# Define the ticker symbol for Nifty 50
ticker_symbol = '^NSEI'

# Define the start and end dates for the data
end_date = date.today()
start_date = end_date - relativedelta(years=2)

# Download historical data
df = yf.download(ticker_symbol, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [5]:
df.reset_index(inplace=True)
df['Day of Week'] = df['Date'].dt.day_name()
df['%_CloseChange'] = df['Close'].pct_change() * 100

In [12]:
# Group by day of the week and calculate average % change
avg_pct_change = df.groupby('Day of Week')['%_CloseChange'].mean()
avg_pct_change

Day of Week
Friday       0.132907
Monday       0.159231
Thursday     0.001800
Tuesday      0.056918
Wednesday    0.041838
Name: %_CloseChange, dtype: float64

In [13]:
df.to_csv("nifty_data.csv")